In [1]:
from   astroplan import moon
from   astropy import time
from   astropy.io import ascii
from   astropy import table
import pandas as pd
import numpy as np

In [ ]:
# URL to the NOT observing calendar

In [ ]:
url = r'http://www.not.iac.es/observing/schedules/schedule.php?period=64&k=&output=html&prn=yes'

In [ ]:
# Retrieve the calendar

In [2]:
tables = pd.read_html(url)
schedule = table.Table.from_pandas(tables[0])
schedule[:5]

Start Date (Noon),End Date (Noon),Proposal Number,Principal Investigator,Institute/ Country,Programme,Instrument,Remarks
str6,str6,str6,str54,str8,str116,str12,str40
Oct 01,Oct 03,64-019,Giorgos Leloudas,DK,Unveiling the nature of blue featureless transients,ALFOSC,--
Oct 03,Oct 04,63-199,Various,NOT,Nordic service time,ALFOSC +FIES,Service
Oct 04,Oct 05,64-199,Various,NOT,Nordic service time,ALFOSC +FIES,Service
Oct 05,Oct 10,64-003,Pasi Hakala,FI,Magnetic accretion and evolution of pre-cataclysmic variables II,ALFOSC,Service
Oct 10,Oct 11,64-199,Various,NOT,Nordic service time,ALFOSC +FIES,Service


In [ ]:
# Identifying the relevant nights

In [3]:
mask_service   = np.where(schedule['Programme'] == 'Nordic service time')[0]
mask_technical = np.where(schedule['Programme'] == 'Technical time')[0]
mask_union     = np.union1d(mask_service, mask_technical)

In [ ]:
# Return the useful observing nights

In [4]:
keys = ['Start Date (Noon)', 'End Date (Noon)', 'Instrument', 'Remarks', 'Programme']
good_nights = schedule[keys][mask_union]
good_nights[-5:]

Start Date (Noon),End Date (Noon),Instrument,Remarks,Programme
str6,str6,str12,str40,str116
Mar 27,Mar 28,ALFOSC +FIES,Service,Nordic service time
Mar 28,Mar 29,FIES +ALFOSC,Remainder of night: Service,Nordic service time
Mar 29,Mar 30,ALFOSC +FIES,Service,Nordic service time
Mar 30,Mar 31,ALFOSC +FIES,Service,Nordic service time
Mar 31,Apr 01,?,--,Technical time


In [ ]:
# Output calendar

In [5]:
year_start = '2021'
year_end   = '2022'

months_year_start = ['Oct', 'Nov', 'Dec']
months_year_end = ['Jan', 'Feb', 'Mar', 'Apr', 'Jun', 'Jul', 'Aug', 'Sep']

month_number = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [6]:
obs_run = table.Table(names=('Start date', 'Start Time', 'End Date', 'End Time', 'Subject'), dtype=('S100', 'S100', 'S100', 'S100', 'S1000'))

for i in range(len(good_nights)):
    
    month, day = good_nights[i]['Start Date (Noon)'].split(' ')
    date       = year_start + '-' + month_number[month] + '-' + day + 'T12:00:00' if month in months_year_start else year_end + '-' + month_number[month]  + '-' + day + 'T12:00:00'
    date       = time.Time(date, format='isot', scale='utc')
    
    tstart     = date
    
    month, day = good_nights[i]['End Date (Noon)'].split(' ')
    date       = year_start + '-' + month_number[month] + '-' + day + 'T12:00:00' if month in months_year_start else year_end + '-' + month_number[month]  + '-' + day + 'T12:00:00'
    date       = time.Time(date, format='isot', scale='utc')
    
    tend       = date
    
    moon_illum = np.round(moon.moon_illumination(tstart, ephemeris=None), 2)
    
    instrument  = good_nights[i]['Instrument']
    remarks     = good_nights[i]['Remarks']
    night_type  = good_nights[i]['Programme']
    
    description = 'Instrument(s): ' + instrument + '; moon illumination: ' + str(moon_illum)
    
    # Fetch the nights when ALFOSC is mounted

    if 'ALFOSC' in instrument:
        #print( tstart, tend, moon_illum, instrument, night_type, remarks)
        obs_run.add_row([tstart.iso.split(' ')[0], tstart.iso.split(' ')[1] + ' PM', tend.iso.split(' ')[0], tend.iso.split(' ')[1] + ' PM', night_type + ' (' + description + ')'])

In [8]:
ascii.write(obs_run, 'NOT_obsrun_P64.csv', format='csv', overwrite=True)

In [9]:
obs_run#[20:]

Start date,Start Time,End Date,End Time,Subject
bytes100,bytes100,bytes100,bytes100,bytes1000
2021-10-03,12:00:00.000 PM,2021-10-04,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.11)
2021-10-04,12:00:00.000 PM,2021-10-05,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.05)
2021-10-10,12:00:00.000 PM,2021-10-11,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.21)
2021-10-11,12:00:00.000 PM,2021-10-12,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.32)
2021-10-14,12:00:00.000 PM,2021-10-15,12:00:00.000 PM,Technical time (Instrument(s): ALFOSC +FIES; moon illumination: 0.65)
2021-10-15,12:00:00.000 PM,2021-10-16,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.75)
2021-10-16,12:00:00.000 PM,2021-10-17,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.83)
2021-10-17,12:00:00.000 PM,2021-10-18,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.9)
2021-10-22,12:00:00.000 PM,2021-10-23,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.97)
